# imports

In [1]:
# general data analysis package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# file management package
import csv, sys
import os
import os.path
import copy
from tqdm.notebook import trange
import warnings
import pickle

# preprocessing onwards
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV, KFold
from sklearn.feature_selection import f_classif

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

from sklearn import svm
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, Normalizer, PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.decomposition import PCA

#from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.inspection import permutation_importance

In [2]:
pwd

'/Users/nadawahyu/Documents/Uni/Master/Forschungspraktikum_2/notebook'

In [3]:
cd ../data

/Users/nadawahyu/Documents/Uni/Master/Forschungspraktikum_2/data


In [4]:
#data_da = pd.read_csv('data_allo_desc.csv')
#data_db = pd.read_csv('data_binding_desc.csv')

data_fa = pd.read_csv('data_allo_fp.csv')#.drop(['Unnamed: 0'], axis=1)
data_fa = data_fa[data_fa['LigandType']!='none']
#data_fb = pd.read_csv('data_binding_fp.csv')#.drop(['Unnamed: 0'], axis=1)
#data_fb = data_fb[data_fb['LigandType']!='none']

In [5]:
# read pickle
#X_train_pa, X_val_pa, X_test_pa, y_train_pa, y_val_pa, y_test_pa, descnm_pa = pd.read_pickle('X_pa.pkl')
#X_train_pb, X_val_pb, X_test_pb, y_train_pb, y_val_pb, y_test_pb, descnm_pb = pd.read_pickle('X_pb.pkl')
#X_train_sa, X_val_sa, X_test_sa, y_train_sa, y_val_sa, y_test_sa, descnm_sa = pd.read_pickle('X_sa.pkl')
#X_train_sb, X_val_sb, X_test_sb, y_train_sb, y_val_sb, y_test_sb, descnm_sb = pd.read_pickle('X_sb.pkl')
#X_train_da, X_val_da, X_test_da, y_train_da, y_val_da, y_test_da, descnm_da = pd.read_pickle('X_da.pkl')
#X_train_db, X_val_db, X_test_db, y_train_db, y_val_db, y_test_db, descnm_db = pd.read_pickle('X_db.pkl')
X_train_fa, X_val_fa, X_test_fa, y_train_fa, y_val_fa, y_test_fa, descnm_fa = pd.read_pickle('X_fa.pkl')
#X_train_fb, X_val_fb, X_test_fb, y_train_fb, y_val_fb, y_test_fb, descnm_fb = pd.read_pickle('X_fb.pkl')

# da

## internal validation

In [6]:
target='LigandType'
col_to_drop=['lig_smiles', 'lig_at']
test_size=0.2
val_size = 0.2
random_state=0

data = data_fa.copy()
data_ = data.copy()

# drop duplicate rows
data_ = data.drop_duplicates().dropna()

# drop cols with 'unnamed'
colidx_to_drop = [i for i in data.columns if 'Unnamed' in i]

#encode target
to_mask = {}

for i,j in enumerate(data_[target].unique()):
  to_mask.update({j:i})

data_ = data_.replace(to_mask)

# separate x and y
X = data_.loc[:, data_.columns.difference(col_to_drop+[target]+colidx_to_drop)]
y = data_[target]

# remove features that don't have variation
X_out = X.loc[:,X.apply(pd.Series.nunique) != 1]
mask = X_out.applymap(lambda x: isinstance(x, (int, float)))
X_out2 = X_out.where(mask)
X = X_out2.dropna(axis=1)

# anova feature selection
f_stat, p_val, = f_classif(X, y)

# get index which probability is lower than p = 0.05
idx = [i for i,j in enumerate(p_val) if j < 0.05]
# select column with respective index
X = X.iloc[:,idx]

# split train-validation and test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size = test_size, random_state=random_state, stratify=y)
# split train and validation
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size = val_size/(1-val_size), random_state=random_state, stratify=y_trainval)

descnm = np.array(X_train.columns)
#X_train = np.array(X_train)
#X_val = np.array(X_val)
#X_test = np.array(X_test)
#y_train = np.array(y_train)
#y_val = np.array(y_val)
#y_test = np.array(y_test)


In [7]:
#X_train, y_train, X_val, y_val = X_train_da, y_train_da, X_val_da, y_val_da
random_state = 0

X_train, y_train  = SMOTE(random_state=random_state).fit_resample(X_train, y_train)

clf = svm.SVC(random_state=random_state)
fit = clf.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.591
f1: 0.575
balanced accuracy: 0.591
matthews corrcoeff: 0.182

val scores 

accuracy: 0.527
f1: 0.667
balanced accuracy: 0.546
matthews corrcoeff: 0.054


In [8]:
print(classification_report(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.63      0.61      2099
           1       0.60      0.55      0.57      2099

    accuracy                           0.59      4198
   macro avg       0.59      0.59      0.59      4198
weighted avg       0.59      0.59      0.59      4198

[[1318  781]
 [ 937 1162]]


In [9]:
print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.11      0.57      0.18        72
           1       0.92      0.52      0.67       700

    accuracy                           0.53       772
   macro avg       0.52      0.55      0.43       772
weighted avg       0.85      0.53      0.62       772

[[ 41  31]
 [334 366]]


## gridsearchcv C only

In [ ]:
param_grid = {
    #'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    #'gamma': ['scale', 'auto'],
    #'class_weight':[None, 'balanced'],
    'C': [100, 10, 1, 0.1, 0.01]
}

In [ ]:
clf = svm.SVC(random_state=random_state)
CV_rfc = GridSearchCV(estimator=clf,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

In [ ]:
clf = svm.SVC(C=100, random_state=random_state)
fit = clf.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.705
f1: 0.711
balanced accuracy: 0.705
matthews corrcoeff: 0.411

val scores 

accuracy: 0.674
f1: 0.794
balanced accuracy: 0.577
matthews corrcoeff: 0.096


In [ ]:
train scores

accuracy: 0.591
f1: 0.575
balanced accuracy: 0.591
matthews corrcoeff: 0.182

val scores

accuracy: 0.527
f1: 0.667
balanced accuracy: 0.546
matthews corrcoeff: 0.054

## gridsearchcv all

### rbf

In [ ]:
param_grid = {
    'kernel': ['rbf',],
    'gamma': ['scale', 'auto'],
    'class_weight':[None, 'balanced'],
    'C': [100, 10, 1, 0.1, 0.01]
}

In [ ]:
clf = svm.SVC(random_state=random_state)
CV_rfc = GridSearchCV(estimator=clf,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV 1/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.671 total time=   2.1s
[CV 2/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.705 total time=   2.3s
[CV 3/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.664 total time=   2.2s
[CV 4/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.695 total time=   2.8s
[CV 5/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.710 total time=   3.2s
[CV 6/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.693 total time=   2.4s
[CV 7/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.705 total time=   2.3s
[CV 8/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.662 total time=   2.2s
[CV 9/10] END C=100, class_weight=None, gamma=scale, kernel=rbf;, score=0.714 total time=   2.2s
[CV 10/10] END C=100, class_weight=None, gamma=scale, kernel=rbf

In [ ]:
clf = svm.SVC(C=100,  random_state=random_state)
fit = clf.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.705
f1: 0.711
balanced accuracy: 0.705
matthews corrcoeff: 0.411

val scores 

accuracy: 0.674
f1: 0.794
balanced accuracy: 0.577
matthews corrcoeff: 0.096


### poly

In [ ]:
param_grid = {
    'kernel': ['poly',],
    #'gamma': ['scale', 'auto'],
    'class_weight':[None, 'balanced'],
    'C': [100, 10, 1, 0.1, 0.01]
}

In [ ]:
clf = svm.SVC(random_state=random_state)
CV_rfc = GridSearchCV(estimator=clf,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END C=100, class_weight=None, kernel=poly;, score=0.629 total time=   3.4s
[CV 2/10] END C=100, class_weight=None, kernel=poly;, score=0.693 total time=   3.6s
[CV 3/10] END C=100, class_weight=None, kernel=poly;, score=0.621 total time=   5.2s
[CV 4/10] END C=100, class_weight=None, kernel=poly;, score=0.638 total time=   3.5s
[CV 5/10] END C=100, class_weight=None, kernel=poly;, score=0.657 total time=   3.5s
[CV 6/10] END C=100, class_weight=None, kernel=poly;, score=0.662 total time=   3.9s
[CV 7/10] END C=100, class_weight=None, kernel=poly;, score=0.688 total time=   4.8s
[CV 8/10] END C=100, class_weight=None, kernel=poly;, score=0.650 total time=   3.3s
[CV 9/10] END C=100, class_weight=None, kernel=poly;, score=0.671 total time=   3.6s
[CV 10/10] END C=100, class_weight=None, kernel=poly;, score=0.656 total time=   3.8s
[CV 1/10] END C=100, class_weight=balanced, kernel=poly;, score=0.629 total time=   4.

In [ ]:
clf = svm.SVC(C=100, kernel='poly', random_state=random_state)
fit = clf.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.668
f1: 0.691
balanced accuracy: 0.668
matthews corrcoeff: 0.340

val scores 

accuracy: 0.698
f1: 0.815
balanced accuracy: 0.547
matthews corrcoeff: 0.061


### sigmoid

In [ ]:
param_grid = {
    'kernel': ['sigmoid',],
    #'gamma': ['scale', 'auto'],
    'class_weight':[None, 'balanced'],
    'C': [100, 10, 1, 0.1, 0.01]
}

In [ ]:
clf = svm.SVC(random_state=random_state)
CV_rfc = GridSearchCV(estimator=clf,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.510 total time=   2.7s
[CV 2/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.495 total time=   2.1s
[CV 3/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.505 total time=   1.8s
[CV 4/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.471 total time=   2.3s
[CV 5/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.483 total time=   2.2s
[CV 6/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.481 total time=   1.5s
[CV 7/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.457 total time=   1.5s
[CV 8/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.457 total time=   1.6s
[CV 9/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.439 total time=   2.0s
[CV 10/10] END C=100, class_weight=None, kernel=sigmoid;, score=0.499 total time=   1.7s
[CV 1/10] END C=100, class_weight=balanced, kernel=sigmo

In [ ]:
clf = svm.SVC(C=0.01, class_weight='balanced', kernel='sigmoid', random_state=random_state)
fit = clf.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.510
f1: 0.115
balanced accuracy: 0.510
matthews corrcoeff: 0.046

val scores 

accuracy: 0.171
f1: 0.160
balanced accuracy: 0.537
matthews corrcoeff: 0.078


### linear

In [10]:
param_grid = {
    'kernel': ['linear',],
    #'gamma': ['scale', 'auto'],
    #'class_weight':[None, 'balanced'],
    'C': [0.01, 0.1, 1, 10, 100],
}

In [11]:
clf = svm.SVC(random_state=random_state)
CV_rfc = GridSearchCV(estimator=clf,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV 1/10] END ............C=0.01, kernel=linear;, score=0.645 total time=44.8min
[CV 2/10] END ............C=0.01, kernel=linear;, score=0.769 total time=29.2min


In [ ]:
clf = svm.SVC(C=0.01, class_weight='balanced', kernel='sigmoid', random_state=random_state)
fit = clf.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.510
f1: 0.115
balanced accuracy: 0.510
matthews corrcoeff: 0.046

val scores 

accuracy: 0.171
f1: 0.160
balanced accuracy: 0.537
matthews corrcoeff: 0.078


## rfecv

In [10]:
svc=svm.SVC(kernel="linear")
rfecv = RFECV(estimator=svc, 
              step=30, 
              cv=KFold(10), # Stratified fold inorder to reduce bias
              scoring='balanced_accuracy', 
              verbose=3)
rfetrain=rfecv.fit(X_train, y_train)
print('Optimal number of features :', rfecv.n_features_)


# Plot showing the Cross Validation score
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


Fitting estimator with 288 features.


## halvinggridsearch

In [11]:
param_grid = {
    'kernel': ['linear',],
    #'gamma': ['scale', 'auto'],
    #'class_weight':[None, 'balanced'],
    'C': [0.01, 0.1, 1, 10, 100],
}

In [14]:
from sklearn.experimental import enable_halving_search_cv

In [15]:
clf = svm.SVC(random_state=random_state)
CV_rfc = model_selection.HalvingGridSearchCV(estimator=clf,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 1399
max_resources_: 4198
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 5
n_resources: 1399
Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV 1/10] END C=0.01, kernel=linear;, score=(train=0.756, test=0.626) total time=15.4min
[CV 2/10] END C=0.01, kernel=linear;, score=(train=0.754, test=0.741) total time=12.1min
[CV 3/10] END C=0.01, kernel=linear;, score=(train=0.745, test=0.647) total time=13.4min
[CV 4/10] END C=0.01, kernel=linear;, score=(train=0.756, test=0.799) total time=17.1min
[CV 5/10] END C=0.01, kernel=linear;, score=(train=0.741, test=0.719) total time=18.5min
[CV 6/10] END C=0.01, kernel=linear;, score=(train=0.732, test=0.719) total time=11.3min
[CV 7/10] END C=0.01, kernel=linear;, score=(train=0.764, test=0.719) total time=10.6min
[CV 8/10] END C=0.01, kernel=linear;, score=(train=0.739, test=0.770) total time=13.4min
[CV 9/10] END C=0.01, kern